# FISTA

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import re

In [ ]:
DATA_PATH = '../../../OneDrive - m.titech.ac.jp/Lab/data'

In [ ]:
def images_to_matrix(folder_path, convert_gray=False, is_binary=False, seed=0, ratio=1.0, use_all=False):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search('random_noise_(\d+).png', f).group(1)))

    if seed != 0:
        random.seed(seed)
        random.shuffle(files)

    total_files = len(files)
    number_of_files_to_load = int(total_files * ratio)

    if use_all:
        selected_files = files[:number_of_files_to_load]
        selected_files.sort(key=lambda f: int(re.search('random_noise_(\d+).png', f).group(1)))
    else:
        # 特定の一部
        selected_indices = []
        a = np.arange(0, 4096).reshape(64, 64)
        a = np.triu(a)
        a = a.flatten().tolist()
        selected_indices = [i for i in a if i != 0]
        selected_indices.insert(0, 0)
        for i in range(0, 64):
            range_i = list(range(63 + 63 * i, 64 * (i + 1)))
            selected_indices += range_i
        selected_files = [files[i] for i in selected_indices if i < total_files]

    images = []
    use_list = []

    for file in selected_files:
        index = int(re.sub(r'\D', '', file))
        use_list.append(index)
        img = Image.open(os.path.join(folder_path, file))
        if convert_gray:
            img = img.convert('L')
        if is_binary:
            img = img.point(lambda x: 255 if x else 0, 'L')
        img_array = np.asarray(img).flatten()
        img_array = img_array / 255
        images.append(img_array)

    # diff = list(set(range(1, len(files)+1))-set(use_list))
    # print("Didn't use:", sorted(diff))
    print("Used:", use_list)

    return np.column_stack(images)


def update_H(H, G, F, lmd, threshold):
    i = 1
    t = 1
    L = max_eigenvalue(F.T @ F) * 3
    print('L: {}'.format(L))
    H_1_prev = H.copy()
    H_2_prev = H.copy()
    while True:
        t_new = (1 + np.sqrt(1 + 4 * t**2)) / 2
        grad = 2 * (H_2_prev @ F - G) @ F.T
        H_1 = soft_thresh(H_2_prev - (1 / L) * grad, lmd / L)
        H_2 = H_1 + (t - 1) / t_new * (H_1 - H_1_prev)
        error = np.linalg.norm(H_2 - H_2_prev, 'fro')

        print('iter: {}, error: {}'.format(i, error))
        if error < threshold:
            break
        t = t_new.copy()
        H_1_prev = H_1.copy()
        H_2_prev = H_2.copy()
        i += 1
    return H_2


def max_eigenvalue(A):
    eigenvalues, eigenvectors = np.linalg.eig(A)
    return np.max(eigenvalues)


def soft_thresh(x, lambda_val):
    return np.sign(x) * np.maximum(np.abs(x) - lambda_val, 0)

In [ ]:
# パラメータ設定
n = 64
m = 128
lmd = 10
threshold = 0.001
seed = 2

In [ ]:
# 真のシステム行列の計算
G_full = images_to_matrix(f"{DATA_PATH}/random_noise{n}_cap/", convert_gray=True, use_all=True)
print(G_full.shape)
# F_full = images_to_matrix(f"{DATA_PATH}/Hadamard{n}_input/", is_binary=True, use_all=True)
# print(F_full.shape)
# H1 = np.zeros((m**2, n**2))
# G1s = []
# for i in range(n**2):
#     G1s.append(G_full[:, 0])
# H1 = np.column_stack(G1s)
# F_hat_full = 2 * F_full - 1
# G_hat_full = 2 * G_full - H1
# H_true = G_hat_full @ F_hat_full.T / (n**2)

In [ ]:
# del F_full
# del F_hat_full
# del G_hat_full
# gc.collect()

In [ ]:
# np.save('../data/systemMatrix/H_matrix_true.npy', H_true)
H_true = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy")

In [ ]:
# 初期化
H = np.zeros((m**2, n**2))

In [ ]:
ratio = 0.5

In [ ]:
G = images_to_matrix(f"{DATA_PATH}/random_noise{n}_cap/", convert_gray=True, seed=seed, ratio=ratio, use_all=True)
F = images_to_matrix(f"{DATA_PATH}/random_noise{n}_input/", convert_gray=True, seed=seed, ratio=ratio, use_all=True)
print(G.shape)
print(F.shape)

In [ ]:
H1 = np.zeros((m**2, 1))
G1s = []
for _ in range(F.shape[1]):
    G1s.append(G_full[:, 0])
H1 = np.column_stack(G1s)
F_hat = 2 * F - 1
G_hat = 2 * G - H1
print(F_hat.shape)
print(G_hat.shape)

In [ ]:
# Hを更新
H = update_H(H, G_hat, F_hat, lmd, threshold)

np.save(f"{DATA_PATH}/systemMatrix/H_matrix_FISTA_rand_{100*ratio}p.npy", H)

# plt.figure(figsize=(12, 8))
# sns.heatmap(H, annot=False, cmap='viridis')

In [ ]:
sample_image = Image.open(f"{DATA_PATH}/sample_image64/Cameraman64.png")
sample_image = sample_image.convert('L')
sample_image = np.asarray(sample_image).flatten() / 255

Hf = H @ sample_image
Hf_img = Hf.reshape(m, m)
Hf_pil = Image.fromarray(np.uint8(Hf_img * 255), mode='L')

DIRECTORY = DATA_PATH + '/240423'
if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)
FILENAME = f"Cameraman64_{ratio}.png"

# plt.figure(figsize=(12, 8))
# sns.heatmap(Hf_img, annot=False, cmap='viridis')

fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_img, cmap='gray')
ax.axis('off')
fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
plt.show()

# fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
# ax.imshow(Hf_pil, cmap='gray')
# ax.axis('off')
# fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
# plt.show()

In [ ]:
rem = np.linalg.norm(H_true-H, 'fro')
print(rem)

In [ ]:
def is_full_rank(matrix):
    rows, cols = matrix.shape
    rank = np.linalg.matrix_rank(matrix)
    
    if rank == min(rows, cols):
        return True
    else:
        return False

print(F_hat.T.shape)
print("F is full rank:", is_full_rank(F_hat.T))